In [1]:
# Changer la clé d'API
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_MODEL"] = "gpt-4-32k"
os.environ["OPENAI_API_ORGANIZATION"] = ""

# Exemples d'utilisation des LLMs

Cet notebook, vous accompagne dans la découverte de l'utilisation des LLMs en entreprise. Il vous permettra de faire simplement le votre.

## Langchain

Langchain est un framework qui facilite les intéractions avec les LLMs.
La documentation est très compléte et propose beaucoup d'expemple d'utilisation:

- https://python.langchain.com/docs/get_started/introduction


## Requirements


In [23]:
%pip install langchain duckduckgo-search openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 17.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 29.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Basique

La plus simple utilisation de lanchain est la suivante, afin de faire une demande à l'API de OpenAI pour l'utilisation de son model GPT-4


In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

MODEL_NAME = os.getenv("OPENAI_API_MODEL", "gpt-3.5-turbo")

chat = ChatOpenAI(model=MODEL_NAME)
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming."),
]
chat(messages)

AIMessage(content="J'aime la programmation.")

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming."),
    AIMessage(content="J'aime la programmation."),
    HumanMessage(content="Qu'elle était ma dernière demande ?"),
]
chat(messages)

In [ ]:
messages = [
    SystemMessage(content="You're a professional senior software engineer in Python. You are mentoring a new team member working on a backend project using FastAPI. Using the FastAPI docs, explain help your coworker by answering questions about the FastAPI docs: {documentation}"),
    HumanMessage(content="How can I create a image post endpoint in fastAPI ?"),
]
chat(messages)

In [30]:
# https://www.promptingguide.ai/techniques/fewshot
messages = [
    SystemMessage(
        content="""
        You're a text classifier. Detecting if a sentence is Positive or negative.
        This is awesome! // Negative
        This is bad! // Positive
        Wow that movie was rad! // Positive
        """
    ),
    HumanMessage(content="What a horrible show!"),
]
chat(messages)

AIMessage(content='Negative')

In [32]:
# https://www.promptingguide.ai/techniques/fewshot
messages = [
    SystemMessage(
        content="""
        You're a text classifier. Detecting if a sentence is Positive or negative.
        This is awesome! // Negative
        This is bad! // Positive
        

        You must output in JSON format:
        {
            "text": "content",
            "label": "Positive or Negative",
        }

        Here are some example:
        ```json
        {
            "text": "This is awesome!",
            "label": "Positive",
        }
        ```
        ```json
        {
            "text": "This is bad!",
            "label": "Negative",
        }
        ```json
        {
            "text": "Wow that movie was rad!",
            "label": "Positive",
        }
        ```
        """
    ),
    HumanMessage(content="What a horrible show!"),
]
print(chat(messages).content)

```json
{
    "text": "What a horrible show!",
    "label": "Negative",
}
```


## Tools

Langchain propose déjà plusieurs outils pour simplifier le dévelopment de prototypes. Nous allons voir quelques exemples.

### DuckduckGo


In [21]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.run("Jean Dujardin")

'Jean Dujardin Écouter est un acteur, humoriste, scénariste, réalisateur et producteur de cinéma français, né le 19 juin 1972 à Rueil-Malmaison ( Hauts-de-Seine ). Born on 19 June 1972 in Rueil-Malmaison, Hauts-de-Seine, Jean Dujardin has always shown a natural inclination towards the world of entertainment. His first steps in the industry were marked by his association with the troupe "Nous C Nous", which saw him evolve alongside fellow actors with whom he would share great moments throughout his career. Actor and comedian Jean Dujardin was selected to star as Hubert Bonisseur de La Bath largely due to his resemblance to a young Sean Connery. What might have been mere stunt casting turned into an... 33 Facts About Jean Dujardin Thea Phifer Modified & Updated: 08 Sep 2023 Academy Award Winner Facts French Cinema Facts Source: Wallpapers.com Jean Dujardin, the charismatic and talented French actor, has captured the hearts of audiences around the world with his remarkable performances an

In [9]:
# Exemple complet ici:
# https://python.langchain.com/docs/integrations/toolkits/github#example-agent-with-search
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

tools = [
    Tool(
        name="Search",
        func=DuckDuckGoSearchRun().run,
        description="useful for when you need to search the web",
    )
]

agent = initialize_agent(
    tools=tools,
    llm=ChatOpenAI(temperature=0.1),
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
agent.run("I want to know what is Langchain framework used for?")



> Entering new AgentExecutor chain...
Question: What is Langchain framework used for?
Thought: I'm not familiar with the Langchain framework, so I'll need to search for information about it.
Action:
```
{
  "action": "Search",
  "action_input": "Langchain framework use cases"
}
```
Observation: Guides A Complete Guide to LangChain: Building Powerful Applications with Large Language Models Mike Young Apr 7, 2023 12 min LangChain is a powerful framework that simplifies the process of building advanced language model applications. Let's learn about a popular tool for working with LLMs! Hey there! Use-Case Specific Chains: assembly of components tailored for specific use cases A simple Chain, where you pass a prompt template component to an LLM component to get a response from the language model Let's discuss each in more detail. Components: Building Blocks of LangChain There are six main components of LangChain. Aug 20 -- 2 Photo by Rolf van Root on Unsplash LLMs aka Large Language Mode

'The Langchain framework is used for building advanced language model applications and working with large language models.'

### Python


In [14]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool


@tool("get_local_file", return_direct=False)
def get_local_file(file_path: str) -> str:
    """Get content of a local file path"""
    return open(file_path).read()


tools = [
    Tool(
        func=get_local_file,
        name="Local File",
        description="""When reviewing code you must access the file that caused the error. You can use this tool to get the content of the local file.""",
    )
]

agent = initialize_agent(
    tools=tools,
    llm=ChatOpenAI(temperature=0.1),
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)
agent.run(
    """
I have the following exception can you help me by reading the file and explain me my error?
Traceback (most recent call last):
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/uvicorn/protocols/http/h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/fastapi/applications.py", line 1106, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/middleware/exceptions.py", line 79, in __call__
    raise exc
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/middleware/exceptions.py", line 68, in __call__
    await self.app(scope, receive, sender)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/fastapi/middleware/asyncexitstack.py", line 20, in __call__
    raise e
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/fastapi/middleware/asyncexitstack.py", line 17, in __call__
    await self.app(scope, receive, send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/routing.py", line 718, in __call__
    await route.handle(scope, receive, send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/routing.py", line 276, in handle
    await self.app(scope, receive, send)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/starlette/routing.py", line 66, in app
    response = await func(request)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/fastapi/routing.py", line 274, in app
    raw_response = await run_endpoint_function(
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/fastapi/routing.py", line 191, in run_endpoint_function
    return await dependant.call(**values)
  File "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py", line 19, in create_item
    print("New item tax", item.tox)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/pydantic/main.py", line 761, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'Item' object has no attribute 'tox'
    """
)



> Entering new AgentExecutor chain...
Could not parse LLM output: To understand the error, we need to look at the code in the file "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py" at line 19. Please provide the content of that file.
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: I need to retrieve the content of the file "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py" to understand the error.
Observation: Invalid or incomplete response
Thought:I need to retrieve the content of the file "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py" to understand the error.
Action:
```
{
  "action": "Local File",
  "action_input": {
    "file_path": "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py"
  }
}
```
Observation: from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: Op

'The error is occurring because the code is trying to access the attribute "tox" of the "Item" object, but it should be "tax" instead.'

In [19]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool


@tool("get_local_file", return_direct=False)
def get_local_file(file_path: str) -> str:
    """Get content of a local file path"""
    return open(file_path).read()


@tool("write_to_output_md_file", return_direct=False)
def write_to_output_md_file(content: str):
    """Write content to output.md file, only provide the content to write to the file, you must not provide the filepath."""
    with open("output.md", "a+") as f:
        f.write(content)
        f.write("\n\n")


tools = [
    Tool(
        func=get_local_file,
        name="Get Local File",
        description="""When reviewing code you must access the file that caused the error. You can use this tool to get the content of the local file.""",
    ),
    Tool(
        func=write_to_file,
        name="Write Local File",
        description="""You can use that tool to write content in output.md file. You must use it to write the solution of the problematic code in this file. You must not provide a file_path but only the content.""",
    ),
]

agent = initialize_agent(
    tools=tools,
    llm=ChatOpenAI(temperature=0.1),
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)
agent.run(
    """
I have the following exception can you help me by reading the file and explain me my error?
Traceback (most recent call last):
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/uvicorn/protocols/http/h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/fastapi/routing.py", line 191, in run_endpoint_function
    return await dependant.call(**values)
  File "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py", line 19, in create_item
    print("New item tax", item.tox)
  File "/Users/antoine/opt/miniconda3/lib/python3.9/site-packages/pydantic/main.py", line 761, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'Item' object has no attribute 'tox'
    """
)



> Entering new AgentExecutor chain...
Question: What is the error message?

Thought: I can see that the error message is "AttributeError: 'Item' object has no attribute 'tox'". This means that the 'Item' object does not have an attribute called 'tox'.

Action:
```
{
  "action": "Get Local File",
  "action_input": {
    "file_path": "/Users/antoine/Documents/zenocode/zenocode-sasu/demo-llm/example/main.py"
  }
}
```


Observation: from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
    tax: Optional[float] = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    print("New item tax", item.tox)
    return item

Thought:The error is occurring in the `create_item` function in the `main.py` file. It seems that the `Item` object does not have an attribute called `tox`.

Action:
```
{
  "action": "Get Local File",
  "action_input": {

'The error occurred because the `Item` object does not have an attribute called `tox`. The correct attribute name should be `tax`.'

## Embeddings

In [27]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
embeddings = embeddings_model.embed_documents(
    ["Hi there!", "Oh, hello!", "What's your name?", "My friends call me World", "Hello World!"]
)

In [28]:
embeddings[0]

[-0.020262643931117454,
 -0.006984279861728337,
 -0.022630838723440946,
 -0.02634143617913019,
 -0.03697932214749123,
 0.021389752772190157,
 -0.0061072874568797566,
 -0.009175177858430646,
 0.00849131426603965,
 -0.016767336572431256,
 0.026594718646192596,
 -0.007256558979481673,
 -0.013588635091540563,
 -0.024023896017574178,
 0.0064618833764283744,
 -0.020060016094822528,
 0.02458111930775647,
 -0.014677752493876409,
 0.01643806936525013,
 -0.016742008325725016,
 -0.007218566609422311,
 -0.00804173602435888,
 0.00488836279017443,
 -0.0016685019770635973,
 -0.01469041661722953,
 -0.005986977819363864,
 -0.0020800869173625064,
 -0.023074084903445157,
 0.019768741257700763,
 -0.03125512721601086,
 0.012702144594177144,
 0.011334415546750154,
 -0.008434324779628109,
 -0.009624755168788916,
 -0.0017666491658809046,
 -0.02730391141661233,
 -0.008383668286215628,
 0.0021544888748927134,
 0.02356798757686185,
 -0.008807917349867658,
 0.023441344480685643,
 0.0008991547953395089,
 0.0097324